In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

### Criando engines de conexão

In [39]:
# Parâmetros de conexão
server = 'localhost'
database = 'usinagem'
username = 'gabriel'
password = 'B13l1201'
driver = 'ODBC Driver 18 for SQL Server'  # ou 18, dependendo do que estiver instalado

# Monta a URL de conexão
conStr = URL.create('mssql+pyodbc', query={'odbc_connect': f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=yes'})

# Cria o engine
engine = create_engine(conStr)


### Importando dados

In [11]:
eixos = pd.read_csv('eixos.csv', sep=';')
notas = pd.read_excel('notas_qm.xlsx')

### Tratando dados dos centros de usinagem

In [20]:
eixos

,dt,maquina,eixo,torque,rBy,rDt,temperatura,TorqMetaFora,TorqMetaEmAtenção,TempMetaFora,TempMetaEmAtenção,Vibracao,Pressao
0,25/10/2022,M00432,B,5.341.228,Fernando Cavalcante Venturoso,21/07/2023 11:32,30.182.116,0,0,0,0,1.145.706,65.346.586
1,25/10/2022,M00432,Magazine,5.182.581,Fernando Cavalcante Venturoso,21/07/2023 11:32,35.393.274,0,0,0,0,1.195.138,65.199.218
2,25/10/2022,M00432,Spindle,7.537.789,Fernando Cavalcante Venturoso,21/07/2023 11:32,102.291.120,0,0,0,0,1.687.219,64.542.681
3,25/10/2022,M00432,X,3.054.814,Fernando Cavalcante Venturoso,21/07/2023 11:32,38.037.017,0,0,0,0,0.720485,64.887.799
4,25/10/2022,M00432,Y,13.675.704,Fernando Cavalcante Venturoso,21/07/2023 11:32,54.596.869,0,0,0,0,2.914.630,64.833.107
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43520,23/08/2025,M00857,Magazine,2.967.329,Node-RED,23/08/2025 08:50,31.757.671,0,0,0,0,0.758995,64.944.350
43521,23/08/2025,M00857,Spindle,28.196.190,Node-RED,23/08/2025 08:50,69.601.981,0,1,0,0,5.828.942,63.883.661
43522,23/08/2025,M00857,X,3.879.549,Node-RED,23/08/2025 08:50,39.021.787,0,0,0,0,0.899432,65.131.074
43523,23/08/2025,M00857,Y,12.920.068,Node-RED,23/08/2025 08:50,46.100.097,0,0,0,0,2.863.676,64.880.216


In [28]:
eixos_reduzido = eixos[['maquina', 'eixo', 'torque','rDt', 'temperatura', 'Vibracao', 'Pressao']]

In [ ]:
eixos_reduzido

Index(['maquina', 'eixo', 'torque', 'rDt', 'temperatura', 'Vibracao',
       'Pressao'],
      dtype='object')

In [41]:
eixos_reduzido.head()

,maquina,eixo,torque,rdt,temperatura,Vibracao,Pressao
0,M00432,B,5.341.228,21/07/2023 11:32,30.182.116,1.145.706,65.346.586
1,M00432,Magazine,5.182.581,21/07/2023 11:32,35.393.274,1.195.138,65.199.218
2,M00432,Spindle,7.537.789,21/07/2023 11:32,102.291.120,1.687.219,64.542.681
3,M00432,X,3.054.814,21/07/2023 11:32,38.037.017,0.720485,64.887.799
4,M00432,Y,13.675.704,21/07/2023 11:32,54.596.869,2.914.630,64.833.107


In [49]:
eixos_reduzido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43525 entries, 0 to 43524
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   maquina      43525 non-null  object
 1   eixo         43525 non-null  object
 2   torque       43525 non-null  object
 3   rdt          43525 non-null  object
 4   temperatura  43525 non-null  object
 5   Vibracao     43525 non-null  object
 6   Pressao      43525 non-null  object
dtypes: object(7)
memory usage: 2.3+ MB


In [33]:
eixos_reduzido.rename(columns={'rDt':'rdt'}, inplace=True)

/var/folders/85/7g55hdxn655gfsc90k47xqkr0000gn/T/ipykernel_13703/75323139.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eixos_reduzido.rename(columns={'rDt':'rdt'}, inplace=True)


In [42]:
colunas = ['torque','temperatura','Vibracao','Pressao']

In [ ]:
for coluna in colunas:
    eixo_tratado = eixos_reduzido

In [43]:
def limpar_numero_especial(valor_str):
    # Verifica se o valor é uma string e tem mais de um ponto
    if isinstance(valor_str, str) and valor_str.count('.') > 1:
        partes = valor_str.split('.')
        # Mantém a primeira parte, o primeiro ponto, e junta o resto
        return partes[0] + '.' + ''.join(partes[1:])
    return valor_str # Retorna o valor original se não corresponder ao padrão

In [44]:
eixos_reduzido[colunas] = eixos_reduzido[colunas].map(limpar_numero_especial)

/var/folders/85/7g55hdxn655gfsc90k47xqkr0000gn/T/ipykernel_13703/1772462365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eixos_reduzido[colunas] = eixos_reduzido[colunas].map(limpar_numero_especial)


In [45]:
eixos_reduzido

,maquina,eixo,torque,rdt,temperatura,Vibracao,Pressao
0,M00432,B,5.341228,21/07/2023 11:32,30.182116,1.145706,65.346586
1,M00432,Magazine,5.182581,21/07/2023 11:32,35.393274,1.195138,65.199218
2,M00432,Spindle,7.537789,21/07/2023 11:32,102.291120,1.687219,64.542681
3,M00432,X,3.054814,21/07/2023 11:32,38.037017,0.720485,64.887799
4,M00432,Y,13.675704,21/07/2023 11:32,54.596869,2.914630,64.833107
...,...,...,...,...,...,...,...
43520,M00857,Magazine,2.967329,23/08/2025 08:50,31.757671,0.758995,64.944350
43521,M00857,Spindle,28.196190,23/08/2025 08:50,69.601981,5.828942,63.883661
43522,M00857,X,3.879549,23/08/2025 08:50,39.021787,0.899432,65.131074
43523,M00857,Y,12.920068,23/08/2025 08:50,46.100097,2.863676,64.880216


In [46]:
eixos_reduzido.to_sql(name='maquinas', con=engine, if_exists='replace')

33

### Tratando Notas de Manutenção

In [51]:
notas

,id,statusSAP,causador,nota_pm,ordem_pm,os_arkmeds,id_os_arkmeds,Unnamed: 7,texto_breve,texto_longo_situacao,...,item_texto,item_texto_causa,texto_acao,historico_texto_longo,tipo_ordem,re,confirmacoes,rBy,rDt,rSt
0,14296,Nota e Ordem PM criada no SAP,-,10768797.0,2365645.0,10726679.0,10702,M2,Equipamento / Maquina não retorna a posição in...,NaN,...,NaN,NaN,NaN,NaN,PM01,501138.0,NaN,Diego Jorge Barbosa da Silva,2025-08-25 15:24:06.270,spnodered
1,14295,Nota e Ordem PM criada no SAP,-,10768796.0,2365644.0,10726678.0,10701,M2,Dispositivo de fixação de peça com defeito de ...,NaN,...,NaN,NaN,NaN,NaN,PM01,501138.0,NaN,Diego Jorge Barbosa da Silva,2025-08-25 15:11:17.420,spnodered
2,14294,Nota e Ordem PM criada no SAP,-,10768795.0,2365643.0,10726677.0,10700,M2,Instalação/ Melhorias/ Alterações,NaN,...,NaN,NaN,NaN,NaN,PM01,501138.0,NaN,Ralph Schaden,2025-08-25 15:05:10.137,spnodered
3,14293,Nota e Ordem PM criada no SAP,-,10768784.0,2365642.0,10726676.0,10699,M2,Banco de teste com defeito,NaN,...,NaN,NaN,NaN,NaN,PM01,501138.0,NaN,Celso Ricardo Vaz Domingues,2025-08-25 14:43:46.577,spnodered
4,14292,Nota e Ordem PM criada no SAP,-,10768783.0,2365641.0,10726675.0,10698,M2,Apertadeira eletrônica com defeito,NaN,...,NaN,NaN,NaN,NaN,PM01,501138.0,NaN,Alex Almeida Pedreira,2025-08-25 14:43:07.877,spnodered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12874,14111,Erro no Sistema Webhook: Campo(s) sem preenchi...,NaN,NaN,NaN,10726587.0,10604,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-08-19 22:50:33.280,spnodered
12875,14107,Erro no Sistema Webhook: Campo(s) sem preenchi...,NaN,NaN,NaN,10726584.0,10601,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-08-19 16:40:31.590,spnodered
12876,14221,Erro no Sistema Webhook: Campo(s) sem preenchi...,NaN,NaN,NaN,10726636.0,10659,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-08-22 10:15:02.727,spnodered
12877,14232,Erro no Sistema Webhook: Campo(s) sem preenchi...,NaN,NaN,NaN,10726646.0,10669,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-08-22 15:45:55.940,spnodered


In [54]:
notas.query('not statusSAP.str.contains("Erro", na=False)')

,id,statusSAP,causador,nota_pm,ordem_pm,os_arkmeds,id_os_arkmeds,Unnamed: 7,texto_breve,texto_longo_situacao,...,item_texto,item_texto_causa,texto_acao,historico_texto_longo,tipo_ordem,re,confirmacoes,rBy,rDt,rSt
0,14296,Nota e Ordem PM criada no SAP,-,10768797.0,2365645.0,10726679.0,10702,M2,Equipamento / Maquina não retorna a posição in...,NaN,...,NaN,NaN,NaN,NaN,PM01,501138.0,NaN,Diego Jorge Barbosa da Silva,2025-08-25 15:24:06.270,spnodered
1,14295,Nota e Ordem PM criada no SAP,-,10768796.0,2365644.0,10726678.0,10701,M2,Dispositivo de fixação de peça com defeito de ...,NaN,...,NaN,NaN,NaN,NaN,PM01,501138.0,NaN,Diego Jorge Barbosa da Silva,2025-08-25 15:11:17.420,spnodered
2,14294,Nota e Ordem PM criada no SAP,-,10768795.0,2365643.0,10726677.0,10700,M2,Instalação/ Melhorias/ Alterações,NaN,...,NaN,NaN,NaN,NaN,PM01,501138.0,NaN,Ralph Schaden,2025-08-25 15:05:10.137,spnodered
3,14293,Nota e Ordem PM criada no SAP,-,10768784.0,2365642.0,10726676.0,10699,M2,Banco de teste com defeito,NaN,...,NaN,NaN,NaN,NaN,PM01,501138.0,NaN,Celso Ricardo Vaz Domingues,2025-08-25 14:43:46.577,spnodered
4,14292,Nota e Ordem PM criada no SAP,-,10768783.0,2365641.0,10726675.0,10698,M2,Apertadeira eletrônica com defeito,NaN,...,NaN,NaN,NaN,NaN,PM01,501138.0,NaN,Alex Almeida Pedreira,2025-08-25 14:43:07.877,spnodered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12724,68,Campo(s) sem preenchimento: Origem do Problema...,Usuário,NaN,NaN,10720292.0,3593,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Flavio Pacifico Muniz,2023-08-16 06:38:28.913,spnodered
12725,70,Campo(s) sem preenchimento: Descrição do Servi...,Usuário,NaN,NaN,10720363.0,3752,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laedson Castanha de Oliveira Santos,2023-08-17 11:55:10.493,spnodered
12726,72,Campo(s) sem preenchimento: Quadro de Trabalho...,Usuário,NaN,NaN,10720365.0,3762,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rogerio Fernando Rodrigues,2023-08-17 14:36:57.590,spnodered
12727,73,Campo(s) sem preenchimento: Origem do Problema...,Usuário,NaN,NaN,10720367.0,3764,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rogerio Fernando Rodrigues,2023-08-17 15:42:51.577,spnodered


In [55]:
notas.statusSAP.value_counts()

statusSAP
Nota e Ordem PM encerrada no SAP                                                                                                                                                        6306
Nota e Ordem PM criada no SAP                                                                                                                                                           6045
Nota e Ordem PM criada e encerrada no SAP                                                                                                                                                 36
Erro na Integração: Documentos alterados e nota encerrada com sucesso. Data(s) real/reais ficam no futuro (verificar parâmetros)                                                          36
Erro no Sistema Webhook: Campo(s) sem preenchimento: Origem do Problema / Quadro de Trabalho / Origem do Problema (Causa) / Descrição do Serviço (Ação).                                  29
                                             

In [56]:
notas.columns

Index(['id', 'statusSAP', 'causador', 'nota_pm', 'ordem_pm', 'os_arkmeds',
       'id_os_arkmeds', 'Unnamed: 7', 'texto_breve', 'texto_longo_situacao',
       'notificador', 'maquina', 'grupo_planejamento',
       'centro_grupo_planejamento', 'centro_trabalho',
       'centro_centro_trabalho', 'prioridade', 'parada', 'inicio_avaria',
       'fim_avaria', 'item_texto', 'item_texto_causa', 'texto_acao',
       'historico_texto_longo', 'tipo_ordem', 're', 'confirmacoes', 'rBy',
       'rDt', 'rSt'],
      dtype='object')

In [70]:
notas_reduzida = notas[['id', 'statusSAP', 'nota_pm', 'texto_breve','maquina', 'grupo_planejamento', 'centro_trabalho', 'inicio_avaria',
       'fim_avaria', 'rDt']]

In [71]:
notas_reduzida

,id,statusSAP,nota_pm,texto_breve,maquina,grupo_planejamento,centro_trabalho,inicio_avaria,fim_avaria,rDt
0,14296,Nota e Ordem PM criada no SAP,10768797.0,Equipamento / Maquina não retorna a posição in...,M01541,210,PC02,2025-08-25 15:24:09.000,NaN,2025-08-25 15:24:06.270
1,14295,Nota e Ordem PM criada no SAP,10768796.0,Dispositivo de fixação de peça com defeito de ...,M00851,210,PC02,2025-08-25 15:11:20.000,NaN,2025-08-25 15:11:17.420
2,14294,Nota e Ordem PM criada no SAP,10768795.0,Instalação/ Melhorias/ Alterações,M10000,200,PC08,2025-08-25 15:05:13.000,NaN,2025-08-25 15:05:10.137
3,14293,Nota e Ordem PM criada no SAP,10768784.0,Banco de teste com defeito,M0BC08,210,PC08,2025-08-25 14:43:50.000,NaN,2025-08-25 14:43:46.577
4,14292,Nota e Ordem PM criada no SAP,10768783.0,Apertadeira eletrônica com defeito,VM15,210,PC08,2025-08-25 14:43:11.000,NaN,2025-08-25 14:43:07.877
...,...,...,...,...,...,...,...,...,...,...
12874,14111,Erro no Sistema Webhook: Campo(s) sem preenchi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-08-19 22:50:33.280
12875,14107,Erro no Sistema Webhook: Campo(s) sem preenchi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-08-19 16:40:31.590
12876,14221,Erro no Sistema Webhook: Campo(s) sem preenchi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-08-22 10:15:02.727
12877,14232,Erro no Sistema Webhook: Campo(s) sem preenchi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-08-22 15:45:55.940


In [ ]:
notas_reduzida = notas_reduzida.query('maquina.notna() and nota_pm.notna()')

In [73]:
notas_reduzida

,id,statusSAP,nota_pm,texto_breve,maquina,grupo_planejamento,centro_trabalho,inicio_avaria,fim_avaria,rDt
0,14296,Nota e Ordem PM criada no SAP,10768797.0,Equipamento / Maquina não retorna a posição in...,M01541,210,PC02,2025-08-25 15:24:09.000,NaN,2025-08-25 15:24:06.270
1,14295,Nota e Ordem PM criada no SAP,10768796.0,Dispositivo de fixação de peça com defeito de ...,M00851,210,PC02,2025-08-25 15:11:20.000,NaN,2025-08-25 15:11:17.420
2,14294,Nota e Ordem PM criada no SAP,10768795.0,Instalação/ Melhorias/ Alterações,M10000,200,PC08,2025-08-25 15:05:13.000,NaN,2025-08-25 15:05:10.137
3,14293,Nota e Ordem PM criada no SAP,10768784.0,Banco de teste com defeito,M0BC08,210,PC08,2025-08-25 14:43:50.000,NaN,2025-08-25 14:43:46.577
4,14292,Nota e Ordem PM criada no SAP,10768783.0,Apertadeira eletrônica com defeito,VM15,210,PC08,2025-08-25 14:43:11.000,NaN,2025-08-25 14:43:07.877
...,...,...,...,...,...,...,...,...,...,...
12691,6,Nota e Ordem PM criada e encerrada no SAP,10747510.0,Defeito na Célula de Carga ou Transdutor de Pr...,M00001,210,PC02,2023-05-16 09:00:00.000,2023-05-17 08:47:00.000,2023-05-17 11:20:56.747
12692,31,Nota e Ordem PM criada e encerrada no SAP,10747877.0,Defeito no CLP,M00001,210,PC02,2023-05-16 06:00:00.000,2023-05-30 07:36:00.000,2023-05-30 07:04:00.680
12694,9,Nota e Ordem PM criada e encerrada no SAP,10747512.0,Agregado - Defeito no Refrigerador (BKW' Refri...,M00452,210,PC02,2023-05-15 16:55:00.000,2023-05-17 16:23:00.000,2023-05-17 16:23:31.683
12696,4,Nota e Ordem PM criada e encerrada no SAP,10747449.0,Defeito no sistema de lavagem,M00470,210,PC02,2023-05-14 06:00:00.000,2023-05-15 08:52:00.000,2023-05-15 12:31:46.903


In [74]:
notas_reduzida.statusSAP.value_counts()

statusSAP
Nota e Ordem PM encerrada no SAP                                                                                                                                                 6306
Nota e Ordem PM criada no SAP                                                                                                                                                    6045
Erro na Integração: Documentos alterados e nota encerrada com sucesso. Data(s) real/reais ficam no futuro (verificar parâmetros)                                                   36
Nota e Ordem PM criada e encerrada no SAP                                                                                                                                          36
Erro na Integração: Nº do documento Arkmeds já foi enviado anteriormente                                                                                                           18
                                                                                

In [75]:
notas_reduzida.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12541 entries, 0 to 12697
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  12541 non-null  int64  
 1   statusSAP           12541 non-null  object 
 2   nota_pm             12541 non-null  float64
 3   texto_breve         12541 non-null  object 
 4   maquina             12541 non-null  object 
 5   grupo_planejamento  12535 non-null  object 
 6   centro_trabalho     12541 non-null  object 
 7   inicio_avaria       12541 non-null  object 
 8   fim_avaria          6468 non-null   object 
 9   rDt                 12541 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 1.1+ MB


In [78]:
notas_reduzida.rename(columns={'rDt':'rdt'}, inplace=True)

/var/folders/85/7g55hdxn655gfsc90k47xqkr0000gn/T/ipykernel_13703/2279260768.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notas_reduzida.rename(columns={'rDt':'rdt'}, inplace=True)


In [100]:
notas_reduzida['nota_pm'] = notas_reduzida['nota_pm'].astype(float).astype('Int64').astype(str)

/var/folders/85/7g55hdxn655gfsc90k47xqkr0000gn/T/ipykernel_13703/1057046853.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notas_reduzida['nota_pm'] = notas_reduzida['nota_pm'].astype(float).astype('Int64').astype(str)


In [88]:
notas_reduzida['id'] = notas_reduzida.id.astype('str')

/var/folders/85/7g55hdxn655gfsc90k47xqkr0000gn/T/ipykernel_13703/1389676885.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notas_reduzida['id'] = notas_reduzida.id.astype('str')


In [101]:
notas_reduzida

,id,statusSAP,nota_pm,texto_breve,maquina,grupo_planejamento,centro_trabalho,inicio_avaria,fim_avaria,rdt
0,14296,Nota e Ordem PM criada no SAP,10768797,Equipamento / Maquina não retorna a posição in...,M01541,210,PC02,2025-08-25 15:24:09.000,NaN,2025-08-25 15:24:06.270
1,14295,Nota e Ordem PM criada no SAP,10768796,Dispositivo de fixação de peça com defeito de ...,M00851,210,PC02,2025-08-25 15:11:20.000,NaN,2025-08-25 15:11:17.420
2,14294,Nota e Ordem PM criada no SAP,10768795,Instalação/ Melhorias/ Alterações,M10000,200,PC08,2025-08-25 15:05:13.000,NaN,2025-08-25 15:05:10.137
3,14293,Nota e Ordem PM criada no SAP,10768784,Banco de teste com defeito,M0BC08,210,PC08,2025-08-25 14:43:50.000,NaN,2025-08-25 14:43:46.577
4,14292,Nota e Ordem PM criada no SAP,10768783,Apertadeira eletrônica com defeito,VM15,210,PC08,2025-08-25 14:43:11.000,NaN,2025-08-25 14:43:07.877
...,...,...,...,...,...,...,...,...,...,...
12691,6,Nota e Ordem PM criada e encerrada no SAP,10747510,Defeito na Célula de Carga ou Transdutor de Pr...,M00001,210,PC02,2023-05-16 09:00:00.000,2023-05-17 08:47:00.000,2023-05-17 11:20:56.747
12692,31,Nota e Ordem PM criada e encerrada no SAP,10747877,Defeito no CLP,M00001,210,PC02,2023-05-16 06:00:00.000,2023-05-30 07:36:00.000,2023-05-30 07:04:00.680
12694,9,Nota e Ordem PM criada e encerrada no SAP,10747512,Agregado - Defeito no Refrigerador (BKW' Refri...,M00452,210,PC02,2023-05-15 16:55:00.000,2023-05-17 16:23:00.000,2023-05-17 16:23:31.683
12696,4,Nota e Ordem PM criada e encerrada no SAP,10747449,Defeito no sistema de lavagem,M00470,210,PC02,2023-05-14 06:00:00.000,2023-05-15 08:52:00.000,2023-05-15 12:31:46.903


In [102]:
notas_reduzida.to_sql(name='notas', con=engine, if_exists='replace')

1

In [80]:
notas.tipo_ordem.value_counts()

tipo_ordem
PM01    12698
Name: count, dtype: int64